In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import os
import plotly.io as pio
pio.templates.default = "plotly_white"
import pandas as pd
from pathlib import Path
from tqdm.autonotebook import tqdm
# %load_ext autoreload
# %autoreload 2
np.random.seed()
tqdm.pandas()

C:\Users\dioni\AppData\Local\Temp\ipykernel_18392\3797367832.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
#Changing the working directory to the root
%cd ../

c:\Users\dioni\Documents\GitHub\Modern-Time-Series-Forecasting-with-Python


-----

# Converting the London Smart Maters dataset into a time series format

## Reading data to pandas dataframe

Let's use London Smart Meters dataset from kaggle for the current exercise. The source of the data is from London Data Store and Jean-Michael D. has augumented the datset with a few more details and packeged it as kaggle dataset

## About the Dataset

The dataset has energy consumption readings for a sample of 5,567 London Households that took part in the UK Power Networks led Low Carbon Londe project between November 2011 and February 2014. Readings were taken at half hourly intervals. Some metadata about the households are also aviable as part of the dataset.

## Data Wrangling

The kagle dataset also has the time series data preprocessed on a daily level and combined all the separate files, etc. But let's ifnore those files and start with the raw files, which is in the hhblock_dataset folder


In [6]:
os.makedirs("imgs/capitulo_2",exist_ok=True)
source_data = Path("data","london_smart_meters")
block_data_path = source_data.joinpath("hhblock_dataset","hhblock_dataset")

In [23]:
block_1 = pd.read_csv(block_data_path/"block_0.csv", parse_dates=False)
block_1['day'] = pd.to_datetime(block_1['day'], yearfirst=True)
block_1.head()

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000002,2012-10-13,0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,...,0.918,0.278,0.267,0.239,0.230,0.233,0.235,0.188,0.259,0.250
1,MAC000002,2012-10-14,0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,...,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.210,0.278,0.159
2,MAC000002,2012-10-15,0.192,0.097,0.141,0.083,0.132,0.070,0.130,0.074,...,1.164,0.249,0.225,0.258,0.260,0.334,0.299,0.236,0.241,0.237
3,MAC000002,2012-10-16,0.237,0.237,0.193,0.118,0.098,0.107,0.094,0.109,...,0.966,0.172,0.192,0.228,0.203,0.211,0.188,0.213,0.157,0.202
4,MAC000002,2012-10-17,0.157,0.211,0.155,0.169,0.101,0.117,0.084,0.118,...,0.223,0.075,0.230,0.208,0.265,0.377,0.327,0.277,0.288,0.256


### Check End Dates of all time series

In [48]:
block_1.groupby("LCLid")["day"].max().sample(5)

LCLid
MAC003252   2014-02-21
MAC003422   2014-02-27
MAC003737   2014-02-27
MAC003805   2014-02-27
MAC003680   2014-02-27
Name: day, dtype: datetime64[ns]

In [51]:
max_date = None
for f in tqdm(block_data_path.glob("*.csv")):
    df = pd.read_csv(f, parse_dates=False)
    df['day'] = pd.to_datetime(df['day'], yearfirst=True)
    if max_date is None:
        max_date = df['day'].max()
    else:
        if df['day'].max()>max_date:
            max_date = df['day'].max()
print(f"Max Date across all blocks: {max_date}")
del df

0it [00:00, ?it/s]

Max Date across all blocks: 2014-02-27 00:00:00


### Reshaping the dataframe into the long form with hour blocks along the rows

In [53]:
block_1.head(2)

,LCLid,day,hh_0,hh_1,hh_2,hh_3,hh_4,hh_5,hh_6,hh_7,...,hh_38,hh_39,hh_40,hh_41,hh_42,hh_43,hh_44,hh_45,hh_46,hh_47
0,MAC000002,2012-10-13,0.263,0.269,0.275,0.256,0.211,0.136,0.161,0.119,...,0.918,0.278,0.267,0.239,0.230,0.233,0.235,0.188,0.259,0.250
1,MAC000002,2012-10-14,0.262,0.166,0.226,0.088,0.126,0.082,0.123,0.083,...,1.075,0.956,0.821,0.745,0.712,0.511,0.231,0.210,0.278,0.159


In [63]:
block_1 = block_1.set_index(['LCLid', "day"]).stack().reset_index().rename(columns={'level_2':'hour_block',0:'energy_consumption'})
block_1['offeset'] = block_1['hour_block'].str.replace("hh_","").astype(int)

In [64]:
block_1.head()

,LCLid,day,hour_block,energy_consumption,offeset
0,MAC000002,2012-10-13,hh_0,0.263,0
1,MAC000002,2012-10-13,hh_1,0.269,1
2,MAC000002,2012-10-13,hh_2,0.275,2
3,MAC000002,2012-10-13,hh_3,0.256,3
4,MAC000002,2012-10-13,hh_4,0.211,4


## Compact Form

In [105]:
start_date = block_1['day'].min()
name = block_1['LCLid'].unique()[0]

dr = pd.date_range(start=start_date, end=max_date, freq='1D')

In [114]:
pd.DataFrame(columns=[f"hh_{i}" for i in range(48)], index=dr).unstack()

hh_0   2011-12-04    NaN
       2011-12-05    NaN
       2011-12-06    NaN
       2011-12-07    NaN
       2011-12-08    NaN
                    ... 
hh_47  2014-02-23    NaN
       2014-02-24    NaN
       2014-02-25    NaN
       2014-02-26    NaN
       2014-02-27    NaN
Length: 39216, dtype: object